In this tutorial, we will show you how to use Haystack to evaluate the performance of a RAG pipeline.

We will use the dataset from the [ARAGOG - Advanced Retrieval Augmented Generation Output Grading (ARAGOG)](https://arxiv.org/pdf/2404.01037) paper.
The dataset is composed of a collection of 13 public AI/LLM-ArXiv research papers and 107 question-answer (QA) pairs. The (QA) pairs generated with the assistance of GPT-4, and then each pair was validated/corrected by humans.

We will use the following Haystack components to evaluate the performance of a RAG pipeline:

- [ContextRelevance](https://docs.haystack.deepset.ai/docs/contextrelevanceevaluator)
- [Faithfulness](https://docs.haystack.deepset.ai/docs/faithfulnessevaluator)
- [Semantic Answer Similarity](https://docs.haystack.deepset.ai/docs/sasevaluator)


We will build a RAG pipeline and then evaluate it using the ARAGOG dataset by varying three parameters:

- `top_k`: the maximum number of documents returned by the retriever
- `embedding_model`: the model used to encode the documents and the question
- `chunk_size`: the number of tokens in the input text that the model can process at once

# Let's first install haystack and the needed dependencies

In [ ]:
%%bash

pip install haystack-ai "git+https://github.com/deepset-ai/haystack.git@main"
pip install "sentence-transformers>=2.2.0"
pip install pypdf
git clone https://github.com/deepset-ai/haystack-evaluation

# Let's define a base path for the dataset

In [43]:
base_path = "haystack-evaluation/datasets/ARAGOG"

## We need to define a function to load the dataset the questions and answers

In [44]:
import json
from typing import Tuple, List

def read_question_answers() -> Tuple[List[str], List[str]]:
    with open(base_path+"/eval_questions.json", "r") as f:
        data = json.load(f)
        questions = data["questions"]
        answers = data["ground_truths"]
    return questions, answers

## We will define an indexing pipeline which depend on two parameters `embedding_model` and the `chunk_size`

In [45]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import PyPDFToDocument
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy

def indexing(embedding_model: str, chunk_size: int):
    files_path = base_path+"/papers_for_questions"
    document_store = InMemoryDocumentStore()
    pipeline = Pipeline()
    pipeline.add_component("converter", PyPDFToDocument())
    pipeline.add_component("cleaner", DocumentCleaner())
    pipeline.add_component("splitter", DocumentSplitter(split_length=chunk_size))  # splitting by word
    pipeline.add_component("writer", DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP))
    pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(embedding_model))
    pipeline.connect("converter", "cleaner")
    pipeline.connect("cleaner", "splitter")
    pipeline.connect("splitter", "embedder")
    pipeline.connect("embedder", "writer")
    pdf_files = [files_path+"/"+f_name for f_name in os.listdir(files_path)]
    pipeline.run({"converter": {"sources": pdf_files}})

    return document_store

## We will define a function to run each query over a RAG architecture.

## Notice that we run the RAG pipeline wrapped within a try/except, to avoid breaking the whole process if at some point there's an error while communicating with OpenAI API.

In [46]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder, AnswerBuilder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators import OpenAIGenerator
from haystack.components.retrievers import InMemoryEmbeddingRetriever


def basic_rag(document_store, embedding_model, top_k=2):
    template = """
        You have to answer the following question based on the given context information only.
        If the context is empty or just a '\n' answer with None, example: "None".

        Context:
        {% for document in documents %}
            {{ document.content }}
        {% endfor %}

        Question: {{question}}
        Answer:
        """

    basic_rag = Pipeline()
    basic_rag.add_component("query_embedder", SentenceTransformersTextEmbedder(
        model=embedding_model, progress_bar=False
    ))
    basic_rag.add_component("retriever", InMemoryEmbeddingRetriever(document_store, top_k=top_k))
    basic_rag.add_component("prompt_builder", PromptBuilder(template=template))
    basic_rag.add_component("llm", OpenAIGenerator(model="gpt-3.5-turbo"))
    basic_rag.add_component("answer_builder", AnswerBuilder())

    basic_rag.connect("query_embedder", "retriever.query_embedding")
    basic_rag.connect("retriever", "prompt_builder.documents")
    basic_rag.connect("prompt_builder", "llm")
    basic_rag.connect("llm.replies", "answer_builder.replies")
    basic_rag.connect("llm.meta", "answer_builder.meta")
    basic_rag.connect("retriever", "answer_builder.documents")

    return basic_rag

In [47]:
from tqdm import tqdm
from pathlib import Path
from openai import BadRequestError

def run_basic_rag(doc_store, sample_questions, embedding_model, top_k):
    """
    A function to run the basic rag model on a set of sample questions and answers
    """

    rag = basic_rag(document_store=doc_store, embedding_model=embedding_model, top_k=top_k)

    predicted_answers = []
    retrieved_contexts = []
    for q in tqdm(sample_questions):
        try:
            response = rag.run(
                data={"query_embedder": {"text": q}, "prompt_builder": {"question": q}, "answer_builder": {"query": q}})
            predicted_answers.append(response["answer_builder"]["answers"][0].data)
            retrieved_contexts.append([d.content for d in response['answer_builder']['answers'][0].documents])
        except BadRequestError as e:
            print(f"Error with question: {q}")
            print(e)
            predicted_answers.append("error")
            retrieved_contexts.append(retrieved_contexts)

    return retrieved_contexts, predicted_answers

## We will define another function to run every predicted answer and ground truth labels through the Evaluators

In [48]:
from haystack.components.evaluators import ContextRelevanceEvaluator, FaithfulnessEvaluator, SASEvaluator
from haystack.evaluation import EvaluationRunResult

def run_evaluation(sample_questions, sample_answers, retrieved_contexts, predicted_answers, embedding_model):
    context_relevance = ContextRelevanceEvaluator(raise_on_failure=False)
    faithfulness = FaithfulnessEvaluator(raise_on_failure=False)
    sas = SASEvaluator(model=embedding_model)
    sas.warm_up()

    results = {
        "context_relevance": context_relevance.run(sample_questions, retrieved_contexts),
        "faithfulness": faithfulness.run(sample_questions, retrieved_contexts, predicted_answers),
        "sas": sas.run(predicted_answers, sample_answers),
    }

    inputs = {'questions': sample_questions, "true_answers": sample_answers, "predicted_answers": predicted_answers}

    return results, inputs

## We also need a function to orchestrate everything, indexing, running the dataset over the RAG for each possible parameter combination, and running the evaluation.
## Notice that for each parameter combination, two `.csv` files are generated:
- one containing the aggregated scores, starting with: `score_report_*`
- another with the full detailed scores for each query, stating with: `detailed_report_*`

In [49]:
def parameter_tuning(questions, answers, out_path: str):
    """
    Run the basic RAG model with different parameters, and evaluate the results.

    The parameters to be tuned are: embedding model, top_k, and chunk_size.
    """
    embedding_models = {
        "sentence-transformers/all-MiniLM-L6-v2",
        "sentence-transformers/msmarco-distilroberta-base-v2",
        "sentence-transformers/all-mpnet-base-v2"
    }
    top_k_values = [1, 2, 3]
    chunk_sizes = [64, 128, 256]

    # create results directory if it does not exist using Pathlib
    out_path = Path(out_path)
    out_path.mkdir(exist_ok=True)

    for embedding_model in embedding_models:
        for chunk_size in chunk_sizes:
            print(f"Indexing documents with {embedding_model} model with a chunk_size={chunk_size}")
            doc_store = indexing(embedding_model, chunk_size)
            for top_k in top_k_values:
                name_params = f"{embedding_model.split('/')[-1]}__top_k:{top_k}__chunk_size:{chunk_size}"
                print(name_params)
                print("Running RAG pipeline")
                retrieved_contexts, predicted_answers = run_basic_rag(doc_store, questions, embedding_model, top_k)
                print(f"Running evaluation")
                results, inputs = run_evaluation(questions, answers, retrieved_contexts, predicted_answers, embedding_model)
                eval_results = EvaluationRunResult(run_name=name_params, inputs=inputs, results=results)
                eval_results.score_report().to_csv(f"{out_path}/score_report_{name_params}.csv",index=False)
                eval_results.to_pandas().to_csv(f"{out_path}/detailed_{name_params}.csv",index=False)

# We can then start the whole process:
  1. Running the RAG pipeline for each query, collecting the generated answers and the retrieved contexts
  2. Call the Haystack Evaluators to compute the performance based on RAG pipeline output and the ground truth data

NOTE: this can take some time due to the parameter search, because of this, we will run the evaluation only for the first 10 questions of the dataset

In [50]:
sample_size = 10
all_questions, all_answers = read_question_answers()
questions = all_questions[0:sample_size]
answers = all_answers[0:sample_size]

In [51]:
questions

['What are the two main tasks BERT is pre-trained on?',
 'What model sizes are reported for BERT, and what are their specifications?',
 "How does BERT's architecture facilitate the use of a unified model across diverse NLP tasks?",
 'Can you describe the modifications LLaMA makes to the transformer architecture for improved performance?',
 "How does LLaMA's approach to embedding layer optimization differ from traditional transformer models, and what are the specific benefits of these modifications?",
 'How were the questions for the multitask test sourced, and what was the criteria for their inclusion?',
 "How does BERT's performance on the GLUE benchmark compare to previous state-of-the-art models?",
 'What significant improvements does BERT bring to the SQuAD v1.1,v2.0 and v13.5 tasks compared to prior models?',
 'What unique aspect of the LLaMA training dataset distinguishes it from datasets used by models like GPT-3, Chinchilla, and PaLM?',
 'What detailed methodology does LLaMA ut

In [52]:
answers

['Masked LM (MLM) and Next Sentence Prediction (NSP).',
 'BERTBASE (L=12, H=768, A=12, Total Parameters=110M) and BERTLARGE (L=24, H=1024, A=16, Total Parameters=340M).',
 'BERT uses a multi-layer bidirectional Transformer encoder architecture, allowing for minimal task-specific architecture modifications in fine-tuning.',
 ' LLaMA incorporates pre-normalization (using RMSNorm), the SwiGLU activation function, and rotary embeddings (RoPE) into its transformer architecture for enhanced training stability and performance.',
 'LLaMA introduces optimizations in its embedding layer through techniques like factorized embeddings and the use of advanced activation functions, such as SwiGLU, for efficient computation and memory usage. These modifications benefit the model by enhancing learning efficiency and enabling the scaling of model parameters without a proportional increase in computational demand.',
 'Questions were manually collected by graduate and undergraduate students from freely av

# We also need to define our OpenAI API credentials

In [18]:
import os
from getpass import getpass
from haystack.components.generators import OpenAIGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
generator = OpenAIGenerator(model="gpt-3.5-turbo")

# Let's start the process

In [19]:
parameter_tuning(questions, answers, "results")

Indexing documents with sentence-transformers/msmarco-distilroberta-base-v2 model with a chunk_size=64


Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82/82 [00:19<00:00,  4.13it/s]


msmarco-distilroberta-base-v2__top_k:1__chunk_size:64
Running RAG pipeline


  0%|                                                                                                                                                                                                                                                 | 0/10 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.34s/it]


Running evaluation


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.95s/it]


all-mpnet-base-v2__top_k:3__chunk_size:256
Running RAG pipeline


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:12<00:00,  1.24s/it]


Running evaluation


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:20<00:00,  2.07s/it]
